In [79]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

In [80]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet

--2023-05-18 11:10:24--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.96, 108.138.245.16, 108.138.245.58, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-01.parquet.1’

yellow_tripdata_202 100%[===================>]  36.37M  51.2MB/s    in 0.7s    

2023-05-18 11:10:25 (51.2 MB/s) - ‘yellow_tripdata_2022-01.parquet.1’ saved [38139949/38139949]



In [81]:
!ls

data                              models
duration-prediction-Copy1.ipynb   yellow_tripdata_2022-01.parquet
duration-prediction.ipynb         yellow_tripdata_2022-01.parquet.1
hw1.ipynb                         yellow_tripdata_2022-02.parquet


In [82]:
df = pd.read_parquet('yellow_tripdata_2022-01.parquet')

In [83]:
df.shape

(2463931, 19)

### Read the data for January. How many columns are there?

Answer: 19

In [84]:
df['duration'] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]

In [85]:
df.dtypes

VendorID                           int64
tpep_pickup_datetime      datetime64[ns]
tpep_dropoff_datetime     datetime64[ns]
passenger_count                  float64
trip_distance                    float64
RatecodeID                       float64
store_and_fwd_flag                object
PULocationID                       int64
DOLocationID                       int64
payment_type                       int64
fare_amount                      float64
extra                            float64
mta_tax                          float64
tip_amount                       float64
tolls_amount                     float64
improvement_surcharge            float64
total_amount                     float64
congestion_surcharge             float64
airport_fee                      float64
duration                 timedelta64[ns]
dtype: object

In [86]:
df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)

In [87]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN,5.966667
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN,10.650000
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN,11.000000
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN,12.050000


In [88]:
df.duration.std()

46.44530513776802

In [89]:
df_no_outliers = df[(df['duration']>=1) & (df['duration']<=60)]

In [90]:
df_no_outliers.shape

(2421440, 20)

In [91]:
df_no_outliers.shape[0]/df.shape[0]

0.9827547930522406

In [92]:
df.dtypes
categorical = ["PULocationID","DOLocationID"]

df_no_outliers[categorical] = df[categorical].astype('str')

/var/folders/cv/xvzlm2fn6kb8b1bk9k_zg7340000gn/T/ipykernel_36497/1426969684.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_outliers[categorical] = df[categorical].astype('str')


In [93]:
# one hot encoding
dv = DictVectorizer()

df_dict = df_no_outliers[categorical].to_dict(orient='records')
X_train = dv.fit_transform(df_dict)
        

In [94]:
dv.feature_names_

['DOLocationID=1',
 'DOLocationID=10',
 'DOLocationID=100',
 'DOLocationID=101',
 'DOLocationID=102',
 'DOLocationID=105',
 'DOLocationID=106',
 'DOLocationID=107',
 'DOLocationID=108',
 'DOLocationID=109',
 'DOLocationID=11',
 'DOLocationID=111',
 'DOLocationID=112',
 'DOLocationID=113',
 'DOLocationID=114',
 'DOLocationID=115',
 'DOLocationID=116',
 'DOLocationID=117',
 'DOLocationID=118',
 'DOLocationID=119',
 'DOLocationID=12',
 'DOLocationID=120',
 'DOLocationID=121',
 'DOLocationID=122',
 'DOLocationID=123',
 'DOLocationID=124',
 'DOLocationID=125',
 'DOLocationID=126',
 'DOLocationID=127',
 'DOLocationID=128',
 'DOLocationID=129',
 'DOLocationID=13',
 'DOLocationID=130',
 'DOLocationID=131',
 'DOLocationID=132',
 'DOLocationID=133',
 'DOLocationID=134',
 'DOLocationID=135',
 'DOLocationID=136',
 'DOLocationID=137',
 'DOLocationID=138',
 'DOLocationID=139',
 'DOLocationID=14',
 'DOLocationID=140',
 'DOLocationID=141',
 'DOLocationID=142',
 'DOLocationID=143',
 'DOLocationID=144',

In [95]:
X_train.shape

(2421440, 515)

In [96]:
target= 'duration'
y_train = df_no_outliers['duration'].values

In [97]:
y_train.shape

(2421440,)

In [98]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [99]:
from sklearn.metrics import mean_squared_error

In [100]:
y_pred = lr.predict(X_train)

In [101]:
mean_squared_error(y_train, y_pred, squared=False)

6.986190769549357

In [102]:
df = pd.read_parquet('yellow_tripdata_2022-02.parquet')

In [103]:
df['duration'] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]

In [104]:
df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)

In [105]:
df = df[(df['duration']>=1) & (df['duration']<=60)]

In [106]:
df_to_dict = df[categorical].to_dict(orient='records')

In [107]:
X_val = dv.transform(df_to_dict)

In [108]:
X_val.shape

(2918187, 515)

In [109]:
y_val = df[target].values

In [110]:
y_pred = lr.predict(X_val)

In [111]:
mean_squared_error(y_val, y_pred, squared=False)

14.619069996595982